In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
!git clone https://github.com/ReallyJaegar/rosetta_neurons.git

In [ ]:
cd rosetta_neurons

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#@title Requirements
!pip install -r requirements.txt
!pip install transformers
!pip install openai-clip
!pip install einops
!pip install pytorch_pretrained_biggan
!pip install Ninja
!pip install timm==0.4.12
!pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git (to revision b46f5ac7587d2e1862f8b7b1573179d80dcdd620) to /tmp/pip-install-4lurmehd/clip_fdda3ea32c954866b5094dff1930e1da
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-4lurmehd/clip_fdda3ea32c954866b5094dff1930e1da
  Running command git rev-parse -q --verify 'sha^b46f5ac7587d2e1862f8b7b1573179d80dcdd620'
  Running command git fetch -q https://github.com/openai/CLIP.git b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Running command git checkout -q b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/arogozhnikov/einops.git (to revision d6f7910dc80c8413b7bb400f6f6ad24e3c5f5b0f) to /tmp/pip-install-4lurmehd/einops_4584e55445d24398b584ca20c

In [ ]:
#@title Imports
from transformers import CLIPProcessor, CLIPModel
import torch
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests
import torch.hub
import time
import pickle
import os
import math
import match_utils
from match_utils import matching, stats, proggan, nethook, dataset, models
device = torch.device('cuda')

In [ ]:
#@title Select Parameters
gan_mode = 'biggan' #@param ['biggan','styleganxl','stylegan2-lsun_cat', 'stylegan2-lsun_horse', 'stylegan2-lsun_car','stylegan3-afhq']
discr_mode = 'dino' #@param ['mae', 'dino_vitb16', 'dino_vitb8', 'dino', 'clip', 'resnet50']
batch_size = 1  #@param {type:"number"}
epochs = 10 #@param {type:"number"}

if "stylegan2" in gan_mode or gan_mode == "stylegan3" or gan_mode == "projgan":
  classidx  = None
else:
  classidx  = 5 #@param {type:"raw"}

if gan_mode == "stylegan3-afhq":
  !gdown "https://drive.google.com/uc?export=download&id=1TWqhCvcAh2Qcu8XQMDoCKwGkqfOFpqL2"
elif gan_mode == "stylegan2-lsun_car":
  !gdown "https://drive.google.com/uc?export=download&id=1kEZG4_-hFVwEYyYwfmXGbJFu9KFSN53s"
elif gan_mode == "stylegan2-lsun_cat":
  !gdown "https://drive.google.com/uc?export=download&id=1nB5RQPrWRfcLI7yHhKKkf4gjxrJW7HCj"
elif gan_mode == "stylegan2-lsun_horse":
  !gdown "https://drive.google.com/uc?export=download&id=16SHYO836iF1AGgMps-EFXP0TxDaXYp3g"

if discr_mode == "mae":
  !gdown "https://drive.google.com/uc?export=download&id=1uJJHiQ6v8Kec9lpG0RXiyeszfGEMOW-6"



save_path = "matches/biggan_clip_497"  #@param {type:"string"}
save_path = "/content/drive/MyDrive/"+save_path
if not os.path.exists(save_path):
  os.makedirs(save_path)

###############################################################

gan, gan_layers = models.load_gan(gan_mode, device)
discr, discr_layers = models.load_discr(discr_mode, device)
z_dataset, c_dataset = dataset.create_dataset(gan, gan_mode, batch_size, epochs, classidx, device)
#Run matching
matching.activ_match_gan(gan = gan, gan_layers = gan_layers,
                         discr = discr, discr_layers = discr_layers,
                         gan_mode = gan_mode,
                         discr_mode = discr_mode,
                         dataset = (z_dataset, c_dataset),
                         epochs = epochs,
                         batch_size = batch_size,
                         save_path = save_path,
                         device = device)




100%|██████████| 715/715 [00:00<00:00, 235080.92B/s]
Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_resnet50_pretrain/dino_resnet50_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_resnet50_pretrain.pth
100%|██████████| 90.0M/90.0M [00:04<00:00, 22.0MB/s]


100%|██████████| 10/10 [00:07<00:00,  1.32it/s]


Finished Iterating for Stats
Done
Starting Activation Matching


100%|██████████| 10/10 [00:39<00:00,  3.96s/it]
